# Run Qwen PPO with [verl](https://github.com/volcengine/verl)

This tutorial provides a step-by-step guide to using veRL for executing your RLHF pipeline. You can find our [github repo](https://github.com/volcengine/verl/) and [documentation](https://verl.readthedocs.io/en/latest/index.html) for mode details.

Please connect to a T4 GPU to run the notebook - It's **free**! However, be aware that the environment is not persisted and may be lost if the session is idle for some time.

### You will learn:

- How to install veRL from scratch.
- How to use existing scripts to run an RLHF pipeline with your own models and data.

# Dependency Installation

If you are running on paperspace, the driver version may not be the latest. We install torch+cu118 for compatibility reasons.

In [1]:
!pip3 uninstall torch torchaudio torchvision tensorflow deepspeed -y
!pip3 install --upgrade pip setuptools wheel
!pip3 install torch==2.4.0 torchvision==0.19.0 --index-url https://download.pytorch.org/whl/cu118
!pip3 list | grep torch
!pip3 install flash-attn --no-build-isolation

Found existing installation: torch 2.1.1+cu121
Uninstalling torch-2.1.1+cu121:
  Successfully uninstalled torch-2.1.1+cu121
Found existing installation: torchaudio 2.1.1+cu121
Uninstalling torchaudio-2.1.1+cu121:
  Successfully uninstalled torchaudio-2.1.1+cu121
Found existing installation: torchvision 0.16.1+cu121
Uninstalling torchvision-0.16.1+cu121:
  Successfully uninstalled torchvision-0.16.1+cu121
Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Successfully uninstalled tensorflow-2.15.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 21.8 MB/s eta 0:00:00
  Attempting uninstall: wheel
    Found existing installation: wheel 0.35.1
    Uninstalling wheel-0.35.1:
      Successfully uninstalled wheel-0.35.1
  Attempting uninstall: setuptools
    Found existing installation: setupt

## Install and verify verl
Now we're ready to install verl!

In [4]:
# In case you run this notebook and have not cloned verl yet:
# !git clone https://github.com/volcengine/verl verl_repo

!cd /notebooks && pip3 install -e . -U

Found existing installation: deepspeed 0.10.3
Uninstalling deepspeed-0.10.3:
  Successfully uninstalled deepspeed-0.10.3
Obtaining file:///notebooks
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for verl (pyproject.toml) ... done
  Created wheel for verl: filename=verl-0.1-0.editable-py3-none-any.whl size=12976 sha256=1efcc6a2eb5d71750e6d1739adf382fd740b2220773d5c0fec96f4e1602d63b7
  Stored in directory: /tmp/pip-ephem-wheel-cache-6nnxs1iw/wheels/92/b1/e0/ce84c2fbc0ed2bc7398e47cccef44e81a78f9916c16dd6e511
Successfully built verl
  Attempting uninstall: verl
    Found existing installation: verl 0.1
    Uninstalling verl-0.1:
      Successfully uninstalled verl-0.1


In [9]:
import torch
try:
  assert torch.cuda.is_available() is True
  torch.ones(1, dtype=torch.bfloat16).cuda()
except AssertionError:
  print("Please start the machine with a GPU supporting bfloat16 (RTX 5000, A5000, A100, H100, A10, etc)")

try:
  import verl
except Exception as e:
  print("Please install verl via pip and restart the kernel")
  raise e

import flash_attn

# Load Pretrained Language Model

verl supports models available in Huggingface transformers (as well as custom Megatron models).

Let's download the model first.

In [4]:
import transformers
transformers.pipeline('text-generation', model='Qwen/Qwen2.5-0.5B-Instruct')

model.safetensors:  58%|#####8    | 577M/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Device set to use cuda:0


# Dataset preparation

We train with the Grade School Math 8K (GSM8k) task in this demo. The dataset is downloaded from huggingface [gsm8k](https://huggingface.co/datasets/openai/gsm8k) and below are some samples:


**Prompt**

Katy makes coffee using teaspoons of sugar and cups of water in the ratio of 7:13. If she used a total of 120 teaspoons of sugar and cups of water, calculate the number of teaspoonfuls of sugar she used.

**Solution**

The total ratio representing the ingredients she used to make the coffee is 7+13 = <<7+13=20>>20 Since the fraction representing the number of teaspoons she used is 7/20, she used 7/20120 = <<7/20120=42>>42 #### 42

In [5]:
!mkdir -p ~/data/gsm8k
!python3 /notebooks/examples/data_preprocess/gsm8k.py --local_dir ~/data/gsm8k

Creating parquet from Arrow format: 100%|████████| 2/2 [00:00<00:00, 332.12ba/s]


# the reward

We use a rule-based reward model. We force the model to produce a final answer following 4 `#` as shown in the solution. We extract the final answer from both the solution and model's output using regular expression matching. We compare them and assign a reward of 1 to correct answer, 0.1 to incorrect answer and 0 to no answer.

In [9]:
import inspect
from verl.utils.reward_score.gsm8k import compute_score as gsm8k_reward
print(inspect.getsource(gsm8k_reward))

def compute_score(solution_str, ground_truth, method='strict', format_score=0., score=1.):
    """The scoring function for GSM8k.

    Reference: Trung, Luong, et al. "Reft: Reasoning with reinforced fine-tuning." Proceedings of the 62nd Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers). 2024.

    Args:
        solution_str: the solution text
        ground_truth: the ground truth
        method: the method to extract the solution, choices are 'strict' and 'flexible'
        format_score: the score for the format
        score: the score for the correct answer
    """
    answer = extract_solution(solution_str=solution_str, method=method)
    if answer is None:
        return 0
    else:
        if answer == ground_truth:
            return score
        else:
            return format_score



# Run the RL Pipeline
Let's start with the Proximal Policy Optimization (PPO) algorithm,  one of the most widely used methods for post-training large language models.

The main entry point of the PPO algorithm example is: `main_ppo.py`. A detailed guide to understanding the code architecture of `main_ppo.py` is available [here](https://verl.readthedocs.io/en/latest/examples/ppo_code_architecture.html).

In this tutorial, we will demonstrate how to run the PPO algorithm with **Qwen 2.5-0.5B** by setting:
- `trainer.n_gpus_per_node`: Number of GPUs per node.

- `actor_rollout_ref.rollout.tensor_model_parallel_size`: TP size for rollout. Only effective for vllm.

- `actor_rollout_ref/critic.model.path`: Huggingface model path. This can be either local path or HDFS path. For HDFS path, we provide utils to download it to DRAM and convert the HDFS path to local path.

- `data.train_batch_size`: Batch size sampled for one training iteration of different RL algorithms.

- `data.max_prompt_length`: Maximum prompt length. All prompts will be left-padded to this length. An error will be reported if the length is too long.

- `data.max_response_length`: Maximum response length. Rollout in RL algorithms (e.g. PPO) generates up to this length.

- `actor_rollout_ref.actor.ppo_mini_batch_size`: One sample is split into multiple sub-batches with batch_size=ppo_mini_batch_size for PPO updates.

- `actor_rollout_ref/critic.actor.ppo_micro_batch_size`: Similar to gradient accumulation, the micro_batch_size for one forward pass, trading speed for GPU memory.

The full configuration explanation is available [here](https://verl.readthedocs.io/en/latest/examples/config.html).

The training may take long time to finish. It will output:

- generated sentences.

- step information with RL metrics.

In [12]:
!python3 -m verl.trainer.main_ppo \
 data.train_files=$HOME/data/gsm8k/train.parquet \
 data.val_files=$HOME/data/gsm8k/test.parquet \
 data.train_batch_size=256 \
 data.val_batch_size=1312 \
 data.max_prompt_length=512 \
 data.max_response_length=256 \
 actor_rollout_ref.model.path=Qwen/Qwen2.5-0.5B-Instruct \
 actor_rollout_ref.actor.optim.lr=1e-6 \
 actor_rollout_ref.actor.ppo_mini_batch_size=64 \
 actor_rollout_ref.actor.ppo_micro_batch_size=1 \
 actor_rollout_ref.rollout.log_prob_micro_batch_size=1 \
 actor_rollout_ref.rollout.tensor_model_parallel_size=1 \
 actor_rollout_ref.rollout.gpu_memory_utilization=0.4 \
 actor_rollout_ref.ref.log_prob_micro_batch_size=1 \
 critic.optim.lr=1e-5 \
 critic.model.path=Qwen/Qwen2.5-0.5B-Instruct \
 critic.ppo_micro_batch_size=1 \
 algorithm.kl_ctrl.kl_coef=0.001 \
 trainer.logger=['console'] \
 +trainer.val_before_train=False \
 trainer.default_hdfs_dir=null \
 trainer.n_gpus_per_node=1 \
 trainer.nnodes=1 \
 trainer.save_freq=10 \
 trainer.test_freq=10 \
 trainer.total_epochs=15 \
 actor_rollout_ref.actor.ppo_micro_batch_size=1 \
 critic.ppo_micro_batch_size=1 \
 actor_rollout_ref.actor.fsdp_config.param_offload=False \
 critic.model.fsdp_config.param_offload=False


2025-01-10 20:02:21,161	ERROR utils.py:535 -- Unexpected error calculating docker cpuset ids.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/ray/_private/utils.py", line 532, in _get_docker_cpus
    cpu_ids.append(int(num_or_range))
                   ^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '\n'
2025-01-10 20:02:21,230	INFO worker.py:1821 -- Started a local Ray instance.
(main_task pid=2286) {'actor_rollout_ref': {'actor': {'clip_ratio': 0.2,
(main_task pid=2286)                                  'entropy_coeff': 0.001,
(main_task pid=2286)                                  'fsdp_config': {'grad_offload': False,
(main_task pid=2286)                                                  'optimizer_offload': False,
(main_task pid=2286)                                                  'param_offload': False,
(main_task pid=2286)                                                  'wrap_policy': {'min_num_params': 0}},
(main_task pid=

# Stop and clean up resources

In [11]:
!ray stop
!nvidia-smi

Did not find any active Ray processes.
Fri Jan 10 20:02:06 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A4000    Off  | 00000000:00:05.0 Off |                  Off |
| 41%   36C    P8    15W / 140W |   2995MiB / 16376MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                 